<a href="https://colab.research.google.com/github/vannisurdi/repVanni/blob/main/EsercizioRecomSyst_Cesarini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Esercizio Recommendation System Cesarini

In [ ]:


####
# Importing libraries
import pandas as pd
import numpy as np


# We'll also import seaborn, a Python graphing library
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")


import matplotlib.pyplot as plt


from sklearn.metrics import classification_report, confusion_matrix


from sklearn.model_selection import train_test_split


from sklearn.svm import LinearSVC # Importing a classifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.model_selection import GridSearchCV # LIbrary for grid search


# Library to compute time measures
import time


from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif,chi2
from sklearn.preprocessing import Binarizer, scale


from sklearn.metrics import roc_auc_score,roc_curve


In [ ]:


####
# Optional library
# This code may benefit the seaborn library. If seaborn is not installed, then follow the next instructions
# run on a code cell the following command
# !pip install seaborn
# If you fail in installing seaborn, don't worry, just skip the few sections where it is used
import seaborn as sns


In [ ]:
#Link to download the dataset
#url='https://www.dropbox.com/s/v7evl2h7svqqey0/train.csv?dl=1'
url='https://drive.google.com/uc?export=download&id=1LqzdqmMwkElgkX_Wpr8PxMWD0xDWIffg'
####
# Loading data
data1 = pd.read_csv(url, index_col='ID') #index_col is the column name containing the record numeric index
#data2 = pd.read_csv('test.csv', index_col='ID') # error_bad_lines=False


In [ ]:

####
# Let's see what's in the trainings data
# The .head() method print the first 5 records and the metadata (e.g., the csv column names)
print('data1.shape')
print(data1.shape)
data1.head() # Last block element doesnt' need the print(), and ve we get also a nicer output


In [ ]:

####
print('Column names')
colNames=data1.columns
#colNames.sort() # let's sort them
print(colNames)
#print(type(colNames)) # <class 'pandas.core.indexes.base.Index'>
print("Number of Columns", len(colNames))
#if you want to see them all, please uncomment below
#print(colNames[0:100]) # From element of index 0 to element of index 99 (100 is the first out)
#print(colNames[100:200])
#print(colNames[200:300])
#print(colNames[300:370])


In [ ]:
### usa il chi quadro e poi l'anova per capire quali variabili spiegano meglio/influenzano il target
# li utilizza in maniera impropria, il risultato potrebbe essere NON statisticam. significativo
# ok lo stesso perché per adesso mi serve solo selezionare le features più importanti!!!
# sto partendo da 370 variabili, devo ridurre il numero prima di cominciare



####
# Feature Selection


# Already imported
# from sklearn.feature_selection import SelectPercentile
# from sklearn.feature_selection import f_classif,chi2
# from sklearn.preprocessing import Binarizer, scale


def selectMostImportantFeatures(matrix,targetFeature, percentile=3):
   # This function selects the top features based on chi2 and f_classif.
   # The matrix parameter is a pandas dataframe containing the feature
   # to be investigated.
   # Two methods are used:
   # * chi2. Chi2 selects the features
   #   which are highly dependent on the response
   #   Best Used for: Categorical which are Boolean,
   #   Frequency and Counts that are non negative
   # * f_classif (ANOVA F-value) for classification features. ANOVA can tell
   #   if an independent variable (e.g. categorical) has significant influence
   #   on Dependent (continuous or ordinal) variable
   # * The percentile parameter is the treshold for selecting the more relevant features 
   # 
   # for the targetFeature classification
   # More info on http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html


   matrix_bin = Binarizer().fit_transform(scale(matrix)) # standardize and binarize
   selectChi2 = SelectPercentile(chi2, percentile=percentile).fit(matrix_bin, targetFeature)
   selectF_classif = SelectPercentile(f_classif, percentile=percentile).fit(matrix, targetFeature)


   chi2_selected = selectChi2.get_support() # boolean values # numpy.ndarray dtype=bool # Get a mask of the features selected
   chi2_selected_features = [ f for i,f in enumerate(matrix.columns) if chi2_selected[i]]
   #print('Chi2 selected {} features {}.'.format(chi2_selected.sum(), chi2_selected_features))


   f_classif_selected = selectF_classif.get_support()
   f_classif_selected_features = [ f for i,f in enumerate(matrix.columns) if f_classif_selected[i]]
   #print('F_classif selected {} features {}.'.format(f_classif_selected.sum(), f_classif_selected_features))
  
   commonSelected = chi2_selected & f_classif_selected # computing a selection mask focusing on the intersection of the two feature sets
   #print('Chi2 & F_classif selected {} features'.format(selected.sum()))
  
   commonFeatures = [ f for f,s in zip(matrix.columns, commonSelected) if s]
   return commonFeatures



In [ ]:


####
target = data1['TARGET']
bestFeature_V1=selectMostImportantFeatures(data1, target)
print(bestFeature_V1)
# TARGET is in the results, this is ok


###scoprirò che il dataset è sbilanciato


In [ ]:
####
# Happy customers have TARGET==0, unhappy custormers have TARGET==1
# From now on: Happy0, Unhappy1
print('Value_counts() result for targetColumn')
print('Remember: Happy0, Unhappy1')
print('Label Count')
print(target.value_counts())


#Let's create a prettier synthetic dataframe (i.e., a new data structure)
df = pd.DataFrame(target.value_counts())
print("Let's rename the TARGET column name to TARGETNUM")
df=df.rename(columns={'TARGET':'TARGETNUM'})
# Adding the Percentage column
recordNum= len(target)
df['Percentage'] = 100.0*df['TARGETNUM']/recordNum
df #The last command of the block is printed even if there is no print()
# Comments are not instructions
# A little less then 4% are unhappy => unbalanced dataset



In [ ]:
### 2 versioni alternative, sotto fanno la stessa cosa

####
# Identifying constant columns
constColNames = []
for col in data1.columns: # Iterating over column names
   if data1[col].std() == 0: # .std() computes the standard deviation over all column values. If .std()== 0 it means the column values are all equals
       constColNames.append(col)
print(constColNames)
print('Total Number of Features', len(data1.columns),'Number of Constant Features', len(constColNames))



####
# A different way to identify constant columns
# Using python list comprehension
constColNames2 = [col for col in data1.columns if data1[col].std() == 0]
print(constColNames2)
print('Total Number of Features', len(data1.columns),'Number of Constant Features', len(constColNames2))

In [ ]:
# Removing constant Columns
data1.drop(constColNames, axis=1, inplace=True) # inplace=True: work inplace, do not create a new (reduced) data structure
# axis=1, 0 is rows, 1 is columns, axis=1 meas: matching column names


In [ ]:
####
# Identify duplicated columns
# i.e., columns that have exactly the same values
duplicateColumns = []
colNames = data1.columns
for i in range(len(colNames)-1): # i ranges from 0 to colNames_length -1
   v = data1[colNames[i]].values # v contains the i_th column values
   for j in range(i+1,len(colNames)):
       # Checking if the column j was not already identified as duplicate
       # and if column j has the same content as column v
       if colNames[j] not in duplicateColumns and np.array_equal(v,data1[colNames[j]].values):
           duplicateColumns.append(colNames[j])
print(duplicateColumns)
print(len(duplicateColumns))


In [ ]:


####
# Removing duplicate columns
data1.drop(duplicateColumns, axis=1, inplace=True) # inplace=True: work inplace, do not create a new (reduced) data structure
# axis=1, 0 is rows, 1 is columns, axis=1 meas: matching column names


In [ ]:


####
# var3 is suspected to be the nationality of the customer
# Don't share this news. It's confidential!!!
# Computing a descriptive statistics of var3
vc=data1['var3'].value_counts()
print('var3 values      frequencies')
print(vc)
print()
print("They are sorted from the most to the less frequent one.")
print("Let's focus on the top 10 most frequent")
print(vc[:10]) #[0:10] or shortly [:10] extracts the first 10 rows




In [ ]:


####
# var3 is suspected to be the nationality of the customer
# Don't share this news. It's confidential!!!
# Computing a descriptive statistics of var3
vc=data1['var3'].value_counts()
print('var3 values      frequencies')
print(vc)
print()
print("They are sorted from the most to the less frequent one.")
print("Let's focus on the top 10 most frequent")
print(vc[:10]) #[0:10] or shortly [:10] extracts the first 10 rows






####
# -999999 it is very likely a null value i.e., the nationality is unknown
# Let's replace this null with the most common value
mask=data1['var3']==-999999 #


# What is mask?
print('mask.shape', mask.shape)
print("data1['var3'].shape", data1['var3'].shape)


print('mask[780:790]')
print(mask[780:790])


print(data1['var3'][780:790])


print("Mask cells are true when the corresponding cells in X1['var3'] are -999999")


# Now using mask to replace -999999 with 2
data1['var3'][mask]=2
print('')
print("Let's have a look to the modified var3 column")
print(data1['var3'][780:790])






####
# Add feature that counts the number of zeros in a row


# I need to exclude TARGET, which is full of zeroes
data1NoTarget = data1.drop(['TARGET'], axis=1)
# data1NoTarget might have been computed in the following way
# data1NoTarget = train.iloc[:,:-1] # target is the last


# The next command performs elementwise the test ...==0
# Then the True values are summed  row-wise (i.e., along axis 1)
# (True is also interpreted as 1)
data1NoTarget['n0'] = (data1NoTarget==0).sum(axis=1)
data1['n0'] = data1NoTarget['n0']


